In [2]:
#!python -m pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.5/522.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.5/230.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.1/930.1 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.3 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for PyMeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=90c97ed91cd8881345cf22ef9dc7af1785732ff22eb9f12e2014a6500706dd06
  Stored in directory: /opt/ml/.cache/pip/wheels/b9/49/19/1ec4bcf2cc76

In [1]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [32]:
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director

In [33]:
director_received_df = pd.read_csv('/opt/ml/input/fighting/FE/director/director_max_sum_intercnt.csv')

In [34]:
director_received_df

,director,dir_max_cnt,dir_sum_cnt
0,nm0000005,1383,6360
1,nm0000019,905,3290
2,nm0000033,5953,36859
3,nm0000036,761,1835
4,nm0000037,2493,2814
...,...,...,...
1336,nm2648685,247,294
1337,nm2676052,906,1563
1338,nm2879822,251,471
1339,nm9054338,1015,1380


In [35]:
director_df.shape
director_df.head(3)

,item,director
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005


In [36]:
# director 결측 item 감독을 'nm0000000'로 채워주기
no_dir_item_list = list(set(main_df['item']) - set(director_df['item']))
no_dir_item_df = pd.DataFrame([x for x in zip(no_dir_item_list, ['nm0000000']*len(no_dir_item_list))])
no_dir_item_df.columns=director_df.columns # 컬럼명 동일하게
director_df = pd.concat([director_df, no_dir_item_df]) # 기존 director_df 뒤에 감독없는 영화 추가

In [37]:
director_df

,item,director
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005
...,...,...
1299,73681,nm0000000
1300,32728,nm0000000
1301,32743,nm0000000
1302,106491,nm0000000


In [38]:
item_list = list(director_df['item'].unique())

In [40]:
df = director_df.merge(director_received_df, how='left', on='director')
df

,item,director,dir_max_cnt,dir_sum_cnt
0,1237,nm0000005,1383,6360
1,5147,nm0000005,1383,6360
2,7327,nm0000005,1383,6360
3,2068,nm0000005,1383,6360
4,7396,nm0000005,1383,6360
...,...,...,...,...
7204,73681,nm0000000,691,691
7205,32728,nm0000000,691,691
7206,32743,nm0000000,691,691
7207,106491,nm0000000,691,691


In [ ]:
# 감독이 'nm0000000'인 경우는 결측치를 채워준 부분이므로, 각 영화를 독립적으로 본다.
# 아래의 코드들은 'nm0000000'인 경우를 처리하기 위한 코드이다.

In [43]:
item_popularity = pd.read_csv('/opt/ml/input/fighting/FE/item/item_interaction_cnt.csv')

In [45]:
df[df['director']=='nm0000000']

,item,director,dir_max_cnt,dir_sum_cnt
5905,40966,nm0000000,691,691
5906,27,nm0000000,691,691
5907,8221,nm0000000,691,691
5908,31,nm0000000,691,691
5909,73759,nm0000000,691,691
...,...,...,...,...
7204,73681,nm0000000,691,691
7205,32728,nm0000000,691,691
7206,32743,nm0000000,691,691
7207,106491,nm0000000,691,691


In [46]:
df = df.merge(item_popularity, how='left', on='item')
df

,item,director,dir_max_cnt,dir_sum_cnt,item_cnt
0,1237,nm0000005,1383,6360,1383
1,5147,nm0000005,1383,6360,949
2,7327,nm0000005,1383,6360,751
3,2068,nm0000005,1383,6360,502
4,7396,nm0000005,1383,6360,335
...,...,...,...,...,...
7204,73681,nm0000000,691,691,58
7205,32728,nm0000000,691,691,50
7206,32743,nm0000000,691,691,48
7207,106491,nm0000000,691,691,74


In [50]:
df.loc[df['director']=='nm0000000', 'dir_max_cnt'] = df.loc[df['director']=='nm0000000']['item_cnt']
df.loc[df['director']=='nm0000000', 'dir_sum_cnt'] = df.loc[df['director']=='nm0000000']['item_cnt']

In [51]:
df

,item,director,dir_max_cnt,dir_sum_cnt,item_cnt
0,1237,nm0000005,1383,6360,1383
1,5147,nm0000005,1383,6360,949
2,7327,nm0000005,1383,6360,751
3,2068,nm0000005,1383,6360,502
4,7396,nm0000005,1383,6360,335
...,...,...,...,...,...
7204,73681,nm0000000,58,58,58
7205,32728,nm0000000,50,50,50
7206,32743,nm0000000,48,48,48
7207,106491,nm0000000,74,74,74


In [52]:
df = df.drop(columns=['item_cnt'], axis=1)

In [53]:
df

,item,director,dir_max_cnt,dir_sum_cnt
0,1237,nm0000005,1383,6360
1,5147,nm0000005,1383,6360
2,7327,nm0000005,1383,6360
3,2068,nm0000005,1383,6360
4,7396,nm0000005,1383,6360
...,...,...,...,...
7204,73681,nm0000000,58,58
7205,32728,nm0000000,50,50
7206,32743,nm0000000,48,48
7207,106491,nm0000000,74,74


In [ ]:
# 이제 각 아이템별 기여한 감독 중 가장 인기많은 감독이 받은 interaction 수를 구할 것이다.

In [54]:
item_grouped = df.groupby('item')

In [56]:
result = item_grouped.max()

In [66]:
result = result.reset_index()
result = result.drop(columns=['director'], axis=1)

In [71]:
result

,item,dir_max_cnt,dir_sum_cnt
0,1,12217,26358
1,2,3364,10935
2,3,1170,2986
3,4,156,268
4,5,766,1749
...,...,...,...
6802,118700,54,54
6803,118900,774,1580
6804,118997,2712,4645
6805,119141,352,474


In [73]:
result.to_csv('item_popular_dir_max_sum_cnt.csv', index=False)